# Advanced decision trees


In this notebook we use Decision Trees for Regression and introduce the idea of ensemble methods through two popular implementations, namely, Random Forests and Gradient Boosting. The three algorithms, Decision Trees for Regression, Random Forests and Gradient Boosting have implementations in PySpark.

## Decision trees for regression

The main difference between Decision Tress for Classification and Decision Trees for Regression is in the impurity measure used. For regression, PySpark uses the variance of the target features as the impurity measure. 

We are going to use the [Wine Quality Dataset](http://archive.ics.uci.edu/ml/datasets/Wine+Quality) to illustrate the use of the **DecisionTreeRegression** class in PySpark. There are twelve input features corresponding to different attributes measured on wine samples (based on physicochemical tests). The target feature corresponds to a quality index that goes from zero to ten being zero a *very bad* wine and ten an *excellent* wine. The target feature was computed as the median score of three independent wine taster experts. More details on the dataset can be found in this [paper](https://www.sciencedirect.com/science/article/pii/S0167923609001377). 

We start by creating a <tt>SparkSession</tt> (unless you are running in a pyspark shell)

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import numpy as np
spark = SparkSession.builder.master("local[2]").appName("COM6012 Decision Trees Regression").getOrCreate()

In [ ]:
rawdata = spark.read.csv('./Data/winequality-white.csv', sep=';', header='true')
rawdata.cache()

Notice that we use the parameter `sep=;` when loading the file, since the columns in the file are separated by `;` instead of the default `,`

In [ ]:
rawdata.printSchema()

We now follow a very familiar procedure to get the dataset to a format that can be input to Spark MLlib, which consists of:
1. transforming the data from type string to type double.
2. to group the features into a type `SparseVector` or `DenseVector`.

We first start transforming the data types.

In [ ]:
schemaNames = rawdata.schema.names
ncolumns = len(rawdata.columns)
from pyspark.sql.types import DoubleType
for i in range(ncolumns):
    rawdata = rawdata.withColumn(schemaNames[i], rawdata[schemaNames[i]].cast(DoubleType()))
rawdata = rawdata.withColumnRenamed('quality', 'labels')

Notice that we used the [<tt>withColumnRenamed</tt>](http://spark.apache.org/docs/2.3.2/api/python/pyspark.sql.html?highlight=withcolumn#pyspark.sql.DataFrame.withColumnRenamed) method to rename the name of the target feature from 'quality' to 'label'. We can now create a DataFrame with the data that we need.

In [ ]:
rawdata.printSchema()

We use the [<tt>VectorAssembler</tt>](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.feature.VectorAssembler) tool to concatenate all the features in a vector.

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = schemaNames[0:ncolumns-1], outputCol = 'features') 
raw_plus_vector = assembler.transform(rawdata)

In [ ]:
data = raw_plus_vector.select('features','labels')

In [ ]:
data.printSchema()

Now that we have the data in the correct format, we can proceed to build the training and test sets.

In [ ]:
(trainingData, testData) = data.randomSplit([0.7, 0.3], 50)

The [DecisionTreeRegressor](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=decisiontreeregress#pyspark.ml.regression.DecisionTreeRegressor) implemented in PySpark has several parameters to tune. Some of them are

> **maxDepth**: it corresponds to the maximum depth of the tree. The default is 5.<p>
**maxBins**: it determines how many bins should be created from continuous features. The default is 32.<p>
    **impurity**: for regression the only supported impurity option is variance.<p>
        **minInfoGain**: it determines the minimum information gain that will be used for a split. The default is zero.


In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(labelCol="labels", featuresCol="features", maxDepth=5)
model = dt.fit(trainingData)
predictions = model.transform(testData)

We finally use the [RegressionEvaluator](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=regressionevaluator#pyspark.ml.evaluation.RegressionEvaluator) tool to assess the rmse on the test set.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator\
      (labelCol="labels", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE = %g " % rmse)

### Question 1

As we did for the Decision Trees for Classification, it is possible to use the [featureImportances](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=featureimportances#pyspark.ml.regression.DecisionTreeRegressionModel.featureImportances) method to study the relative importance of each features. Use the *featureImportances* in the model above and indicate the three most relevant features.

### Question 2

Write a pipeline that includes a parameter grid that allows you to find the best parameter configuration for the parameters *maxDepth* and *maxBins*.

## Ensemble learning

In machine learning, we use the term ensemble model to refer to a predictive model that is a composition of several other predictive models. For example, for a classification problem, we can have an ensemble of three classifiers, where the first of them is a Naive Bayes classifier, the second one is a logistic regressor and the third one is a decision tree classifier. We can train all classifiers with the same training data and then, at test time, predictions can be done using majority voting. 

Ensemble methods are very popular since they usually show higher performance when compared to simple classifiers. In fact, gradient boosting trees are the most popular method in [**Kaggle**](https://www.kaggle.com/), a platform that hosts data science competitions. The top entry in the [**Netflix Prize**](https://en.wikipedia.org/wiki/Netflix_Prize) Competition, one of the most famous data science competitions, was based on an ensemble predictive model. 

The most commmon ensemble methods use decision trees as the members of the ensemble. PySpark implemenst two types of Tree Ensembles, random forests and gradient boosting. The main difference between both methods is the way in which they combine the different trees that compose the ensemble.

### Random Forests

The variant of Random Forests implemented in Apache Spark is also known as bagging or boostrap aggregating. The tree ensemble in random forests is built by training individual decision trees on different subsets of the training data and using a subset of the available features. For classification, the prediction is done by majority voting among the individual trees. For regression, the prediction is the average of the individual predictions of each tree. For more details on the PySpark implmentation see [here](http://spark.apache.org/docs/2.3.2/mllib-ensembles.html#random-forests). 

Besides the parameters that we already mentioned for the [DecisionTreeClassifier](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=decisiontreeclassifier#pyspark.ml.classification.DecisionTreeClassifier) and the [DecisionTreeRegressor](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=decisiontreeregress#pyspark.ml.regression.DecisionTreeRegressor), the [RandomForestClassifier](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=randomforestclassifier#pyspark.ml.classification.RandomForestClassifier) and the [RandomForestRegressor](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=randomforestregressor#pyspark.ml.regression.RandomForestRegressor) in PySpark require three additional parameters:
> **numTrees** the total number of trees to train<p>
**featureSubsetStrategy** number of features to use as candidates for splitting at each tree node. Options include all, onethird, sqrt, log2, [1-n]<p>
    **subsamplingRate**: size of the dataset used for training each tree in the forest, as a fraction of the size of the original dataset. 

Let us use RandomForestRegressor on the wine quality dataset from above and evaluate the performance on the same data set partition that we had before.


In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rfr = RandomForestRegressor(labelCol="labels", featuresCol="features", maxDepth=5, numTrees=3)
model = rfr.fit(trainingData)
predictions = model.transform(testData)

from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator\
      (labelCol="labels", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE = %g " % rmse)

We can also use [featuresImportance](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=featureimportances#pyspark.ml.regression.RandomForestRegressionModel.featureImportances) for the RandomForestRegressor and RandomForestClassifier models. How are the feature importances computed in this case? 

### Question 3

Write a pipeline that includes a parameter grid that allows you to find the best parameter configuration for the parameters *maxDepth*, *maxBins*, *numTrees*, *featureSubsetStrategy* and *subsamplingRate*.

### Gradient Boosting

In [Gradient Boosting](https://en.wikipedia.org/wiki/Gradient_boosting) or [Gradient-boosted trees](https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting) (GBT), each tree in the ensemble is trained sequentially: the first tree is trained as usual using the training data, the second tree is trained on the residuals between the predictions of the first tree and the labels of the training data, the third tree is trained on the residuals of the predictions of the second tree, etc. The predictions of the ensemble will be the sum of the predictions of each individual tree. The type of residuals are related to the loss function that wants to be minimised. In the PySpark implementations of Gradient-Boosted trees, the loss function for binary classification is the Log-Loss function and the loss function for regression is either the squared error or the absolute error. For details, follow this [link](http://spark.apache.org/docs/2.3.2/mllib-ensembles.html#gradient-boosted-trees-gbts).  

PySpark uses the classes [GBTRegressor](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=gradient%20boosting#pyspark.ml.regression.GBTRegressor) for the implementation of Gradient-Boosted trees for regression and [GBTClassifier](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=gbtclassifier#pyspark.ml.classification.GBTClassifier) for the implementation of Gradient-Boosted trees for binary classification. As of PySpark version 2.3.2 GBT have not been implemented for multiclass classification.

Besides the parameters that can be specified for Decision Trees, both classes share the additional following parameters

>**lossType** type of loss function. Options are "squared" and "absolute" for regression and "logistic" for classification. <p>
    **maxIter** number of trees in the ensemble. Each iteration produces one tree.<p>
    **stepSize** also known as the learning rate, it is used for shrinking the contribution of each tree in the sequence. The default is 0.1<p>
    **subsamplingRate** as it was the case for Random Forest, this parameter is used for specifying the fraction of the training data used for learning each decision tree.    

We will now use the GBTRegressor on the wine quality dataset.

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbtr = GBTRegressor(labelCol="labels", featuresCol="features", maxDepth=5, maxIter=5, lossType='squared')
model = gbtr.fit(trainingData)
predictions = model.transform(testData)

from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator \
      (labelCol="labels", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE = %g " % rmse)

### Question 4

Write and run an HPC standalone program using random forest regression on the [Physical Activity Monitoring](http://archive.ics.uci.edu/ml/datasets/PAMAP2+Physical+Activity+Monitoring) dataset, methodically experimenting with the parameters *maxDepth*, *numTrees* and *subsamplingRate*. Obtain the timing for the experiment. Note that the <tt>physical activity monitoring</tt> dataset contains <tt>NaN</tt> (not a number) values when values are missing - you should try dealing with this in two ways

1. Drop lines containing <tt>NaN</tt>
2. Replace <tt>NaN</tt> with the average value from that column. For this, you can use the [Imputer](http://spark.apache.org/docs/2.3.2/ml-features.html#imputer) transformer available in <tt>pyspark.ml.feature</tt> 

Run experiments with both options.